In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/CoLeaf DATASET (1).zip"

Archive:  /content/drive/MyDrive/CoLeaf DATASET (1).zip
  inflating: CoLeaf DATASET/healthy.zip  
  inflating: CoLeaf DATASET/iron-Fe.zip  
  inflating: CoLeaf DATASET/boron-B.zip  
  inflating: CoLeaf DATASET/magnesium-Mg.zip  
  inflating: CoLeaf DATASET/nitrogen-N.zip  
  inflating: CoLeaf DATASET/manganese-Mn.zip  
  inflating: CoLeaf DATASET/more-deficiencies.zip  
  inflating: CoLeaf DATASET/calcium-Ca.zip  
  inflating: CoLeaf DATASET/potasium-K.zip  
  inflating: CoLeaf DATASET/phosphorus-P.zip  


In [ ]:
!mkdir "Dataset"

In [ ]:
!unzip "/content/CoLeaf DATASET/boron-B.zip" -d "/content/Dataset"
!unzip "/content/CoLeaf DATASET/calcium-Ca.zip" -d "/content/Dataset"
!unzip "/content/CoLeaf DATASET/healthy.zip" -d "/content/Dataset"
!unzip "/content/CoLeaf DATASET/iron-Fe.zip" -d "/content/Dataset"
!unzip "/content/CoLeaf DATASET/magnesium-Mg.zip" -d "/content/Dataset"
!unzip "/content/CoLeaf DATASET/manganese-Mn.zip" -d "/content/Dataset"
!unzip "/content/CoLeaf DATASET/more-deficiencies.zip" -d "/content/Dataset"
!unzip "/content/CoLeaf DATASET/nitrogen-N.zip" -d "/content/Dataset"
!unzip "/content/CoLeaf DATASET/phosphorus-P.zip" -d "/content/Dataset"
!unzip "/content/CoLeaf DATASET/potasium-K.zip" -d "/content/Dataset"






Archive:  /content/CoLeaf DATASET/boron-B.zip
   creating: /content/Dataset/boron-B/
  inflating: /content/Dataset/boron-B/B (1).jpg  
  inflating: /content/Dataset/boron-B/B (10).jpg  
  inflating: /content/Dataset/boron-B/B (100).jpg  
  inflating: /content/Dataset/boron-B/B (101).jpg  
  inflating: /content/Dataset/boron-B/B (11).jpg  
  inflating: /content/Dataset/boron-B/B (12).jpg  
  inflating: /content/Dataset/boron-B/B (13).jpg  
  inflating: /content/Dataset/boron-B/B (14).jpg  
  inflating: /content/Dataset/boron-B/B (15).jpg  
  inflating: /content/Dataset/boron-B/B (16).jpg  
  inflating: /content/Dataset/boron-B/B (17).jpg  
  inflating: /content/Dataset/boron-B/B (18).jpg  
  inflating: /content/Dataset/boron-B/B (19).jpg  
  inflating: /content/Dataset/boron-B/B (2).jpg  
  inflating: /content/Dataset/boron-B/B (20).jpg  
  inflating: /content/Dataset/boron-B/B (21).jpg  
  inflating: /content/Dataset/boron-B/B (22).jpg  
  inflating: /content/Dataset/boron-B/B (23).jpg

In [ ]:

# Import necessary libraries
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define image dimensions
img_height, img_width = 224, 224
batch_size = 32

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # 80% training, 20% validation

train_generator = train_datagen.flow_from_directory(
    '/content/Dataset',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    '/content/Dataset',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

# Determine number of training and validation samples
num_train_samples = len(train_generator.filenames)
num_val_samples = len(validation_generator.filenames)

# EfficientNet
base_model1 = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
x1 = base_model1.output
x1 = GlobalAveragePooling2D()(x1)
x1 = Dense(1024, activation='relu')(x1)
predictions1 = Dense(len(train_generator.class_indices), activation='softmax')(x1)
model1 = Model(inputs=base_model1.input, outputs=predictions1)

# Freeze pre-trained layers
for layer in base_model1.layers:
    layer.trainable = False

# Compile the model
model1.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history1 = model1.fit(train_generator,
                      steps_per_epoch=num_train_samples // batch_size,
                      epochs=10,
                      validation_data=validation_generator,
                      validation_steps=num_val_samples // batch_size)

# Evaluate the model
evaluation1 = model1.evaluate(validation_generator)
print("Evaluation (EfficientNet):", evaluation1)


Found 809 images belonging to 10 classes.
Found 197 images belonging to 10 classes.
16705208/16705208 [==============================] - 1s 0us/step
Epoch 1/10
25/25 [==============================] - 116s 4s/step - loss: 2.2437 - accuracy: 0.1875 - val_loss: 2.1730 - val_accuracy: 0.1615
Epoch 2/10
25/25 [==============================] - 93s 4s/step - loss: 2.1930 - accuracy: 0.2239 - val_loss: 2.1791 - val_accuracy: 0.2500
Epoch 3/10
25/25 [==============================] - 89s 4s/step - loss: 2.1830 - accuracy: 0.2072 - val_loss: 2.1668 - val_accuracy: 0.2500
Epoch 4/10
25/25 [==============================] - 89s 4s/step - loss: 2.1546 - accuracy: 0.2471 - val_loss: 2.1273 - val_accuracy: 0.2552
Epoch 5/10
25/25 [==============================] - 90s 4s/step - loss: 2.1768 - accuracy: 0.2162 - val_loss: 2.1586 - val_accuracy: 0.1667
Epoch 6/10
25/25 [==============================] - 91s 4s/step - loss: 2.1478 - accuracy: 0.2252 - val_loss: 2.1205 - val_accuracy: 0.2552
Epoch 7/10

In [6]:
# Import necessary libraries
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Data generators (assuming the dataset is already extracted)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # 80% training, 20% validation

train_generator = train_datagen.flow_from_directory(
    '/content/Dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    '/content/Dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation') # set as validation data

# Determine number of training and validation samples
num_train_samples = len(train_generator.filenames)
num_val_samples = len(validation_generator.filenames)

# MobileNetV2
base_model2 = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x2 = base_model2.output
x2 = GlobalAveragePooling2D()(x2)
x2 = Dense(1024, activation='relu')(x2)
predictions2 = Dense(len(train_generator.class_indices), activation='softmax')(x2)
model2 = Model(inputs=base_model2.input, outputs=predictions2)

# Freeze pre-trained layers
for layer in base_model2.layers:
    layer.trainable = False

# Compile the model
model2.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history2 = model2.fit(train_generator,
                      steps_per_epoch=num_train_samples // 32,
                      epochs=10,
                      validation_data=validation_generator,
                      validation_steps=num_val_samples // 32)

# Evaluate the model
evaluation2 = model2.evaluate(validation_generator)
print("Evaluation (MobileNetV2):", evaluation2)


Found 809 images belonging to 10 classes.
Found 197 images belonging to 10 classes.
9406464/9406464 [==============================] - 1s 0us/step
Epoch 1/10
25/25 [==============================] - 120s 5s/step - loss: 1.7807 - accuracy: 0.4479 - val_loss: 1.7855 - val_accuracy: 0.4740
Epoch 2/10
25/25 [==============================] - 102s 4s/step - loss: 0.9988 - accuracy: 0.6486 - val_loss: 1.6529 - val_accuracy: 0.4896
Epoch 3/10
25/25 [==============================] - 93s 4s/step - loss: 0.8422 - accuracy: 0.7150 - val_loss: 1.4997 - val_accuracy: 0.5260
Epoch 4/10
25/25 [==============================] - 91s 4s/step - loss: 0.6967 - accuracy: 0.7259 - val_loss: 1.5667 - val_accuracy: 0.5417
Epoch 5/10
25/25 [==============================] - 112s 4s/step - loss: 0.6564 - accuracy: 0.7503 - val_loss: 1.7636 - val_accuracy: 0.5365
Epoch 6/10
25/25 [==============================] - 113s 5s/step - loss: 0.6905 - accuracy: 0.7516 - val_loss: 1.6396 - val_accuracy: 0.5312
Epoch 7/1